In [288]:
import numpy as np
import pandas as pd

In [289]:
df = pd.read_csv('../data/processed/msk_2018_2021.csv')

In [290]:
district = 'Тверской район'
object_type = 1
rooms = 2

In [291]:
df = df[df['district'] == district]
df = df[df['object_type'] == object_type]
df = df[df['rooms'] == rooms]
df.reset_index(drop = True, inplace = True)
# & df['object_type'] == object_type & df['rooms'] == rooms#3

In [292]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'price', 'date', 'time', 'geo_lat',
       'geo_lon', 'building_type', 'level', 'levels', 'rooms', 'area',
       'kitchen_area', 'object_type', 'timestamp', 'target', 'year', 'month',
       'distance_from_centre', 'district'],
      dtype='object')

In [293]:
def drop(df : pd.DataFrame):
    df.drop('date', axis=1, inplace=True)
    df.drop('geo_lat', axis=1, inplace=True)
    df.drop('geo_lon', axis=1, inplace=True)
    df.drop('building_type', axis=1, inplace=True)
    df.drop('level', axis=1, inplace=True)
    df.drop('levels', axis=1, inplace=True)
    df.drop('rooms', axis=1, inplace=True)
    df.drop('object_type', axis=1, inplace=True)
    df.drop('kitchen_area', axis=1, inplace=True)
    df.drop('distance_from_centre', axis=1, inplace=True)
    df.drop('district', axis=1, inplace=True)
    df.drop('Unnamed: 0.1', axis=1, inplace=True)
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df.drop('price', axis=1, inplace=True)
    df.drop('time', axis=1, inplace=True)
    df.drop('area', axis=1, inplace=True)


In [294]:
def price_metr(df : pd.DataFrame):
    df['target'] = df['price'] / df['area']

In [295]:
price_metr(df)
drop(df)

In [296]:
df.head(50)
print(df.shape)

(2273, 4)


In [297]:
df.tail()

,timestamp,target,year,month
2268,2021-05-01,228181.818182,2021,5
2269,2021-05-01,217727.272727,2021,5
2270,2021-05-01,242222.222222,2021,5
2271,2021-05-01,455217.391304,2021,5
2272,2021-05-01,226651.515152,2021,5


In [298]:
df['target'].quantile(0.9)

387052.13738571777

Деление на бизнес и эконом каждый месяц

In [299]:
start_year = df['year'].min()
month_start = df.loc[df['year'] == start_year]['month'].min()
stop_year = df['year'].max()
month_end = df.loc[df['year'] == stop_year]['month'].max()

In [300]:
df_cat_econom = pd.DataFrame(columns = ['timestamp', 'target', 'size' ])
df_cat_bisnes = pd.DataFrame( columns = ['timestamp', 'target', 'size' ])
df_category = pd.DataFrame( columns = ['timestamp', 'target' , 'size'])

In [301]:
for year in range(start_year,stop_year+1):
    if year == start_year:
        start_month = month_start
    else:
        start_month = 1
    if year == stop_year:
        stop_month = month_end
    else:
        stop_month = 12
    for month in range(start_month,stop_month+1):
        # Вырезаем данный месяц из исходного датасета
        df_category_add = df.loc[(df['month'] == month) & (df['year'] == year)].copy()
        # Удаляем уже не нежные столбцы
        df_category_add.drop('year', axis=1, inplace=True)
        df_category_add.drop('month', axis=1, inplace=True)
        # обрезаем выбромы (хвостики) из данного месяцы
        df_category_add = df_category_add[(df_category_add['target'] > df_category_add['target'].quantile(0.25)) & (df_category_add['target'] < df_category_add['target'].quantile(0.9))]
        df_category = pd.concat([df_category,pd.DataFrame({'timestamp' : [f"{year}-{month}-01"] , 'target' : [df_category_add['target'].median()] , 'size': [df_category_add.shape[0]] })])
        # определяем две новые категории эконом и бизнес
        df_cat_econom_add = df_category_add[df_category_add['target'] < df_category_add['target'].median()].copy()
        df_cat_bisnes_add = df_category_add[df_category_add['target'] > df_category_add['target'].median()].copy()
        # Добавляем данный месяц в итоговые датасеты эконом и бизнес
        df_cat_econom = pd.concat([df_cat_econom,pd.DataFrame({'timestamp' : [f"{year}-{month}-01"] , 'target' : [df_cat_econom_add['target'].median()], 'size': [df_cat_econom_add.shape[0]]})])
        df_cat_bisnes = pd.concat([df_cat_bisnes,pd.DataFrame({'timestamp' : [f"{year}-{month}-01"] , 'target' : [df_cat_bisnes_add['target'].median()], 'size': [df_cat_bisnes_add.shape[0]]})])

C:\Users\user\AppData\Local\Temp\ipykernel_11700\4153855823.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_category = pd.concat([df_category,pd.DataFrame({'timestamp' : [f"{year}-{month}-01"] , 'target' : [df_category_add['target'].median()] , 'size': [df_category_add.shape[0]] })])
C:\Users\user\AppData\Local\Temp\ipykernel_11700\4153855823.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_cat_econom = pd.concat([df_cat_econom,pd.DataFrame({'timestamp' : [f"{year}-{month}-01"] , 

In [302]:
df_cat_econom.reset_index(drop = True, inplace = True)


In [303]:
df_cat_bisnes.reset_index(drop = True, inplace = True)

In [304]:
df_category.reset_index(drop = True, inplace = True)

In [305]:
df_category.drop('size', axis=1, inplace = True)
df_category.to_csv("../data/serial_category/seria_test.csv")